In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from ortools.linear_solver import pywraplp

# Defining data inputs

## Defining number of PTO days

This one is easy but must be defined somewhere: the total number of PTO days we get in a year.  We'll store this in a variable:

In [2]:
D_PTO = 20

Here, I set our number of PTO days to 20 but it can be set to anything that is applicable to you.

## Defining given days off

We need to identify the days in the year that we get off for "free", i.e. days where we wouldn't have to spend a PTO day.  These freedays could include federal/bank/company holidays and weekends.  Let's start by creating a pandas dataframe of all the days of the year:

In [3]:
yearSeries = pd.date_range('2023-01-01', '2023-12-31', freq='D').to_series()
yearDF = pd.DataFrame(yearSeries, columns = ['Day'])
yearDF['Day of Year'] = yearDF['Day'].dt.dayofyear

In yearDF, each row is a day of the year, and we have a column for the date of that day ("Day") and what day of the year that day is ("Day of Year").

Next, we can create a new column called "Freeday" that will act as a flag if a particular day is a freeday.  We start below with a list of the days we have free as company holidays, then set their value in the "Freeday" column of the dataframe as true:

In [4]:
company_holidays = [
    '2023-01-02',
    '2023-01-16',
    '2023-04-07',
    '2023-04-10',
    '2023-05-29',
    '2023-07-03',
    '2023-07-04',
    '2023-09-04',
    '2023-11-10',
    '2023-11-23',
    '2023-11-24',
    '2023-12-25',
    '2023-12-26',
    '2023-12-27',
    '2023-12-28',
    '2023-12-29',
]

yearDF['Freeday'] = yearDF['Day'].isin(pd.to_datetime(company_holidays))

These are example days and should be changed to whatever is applicable to you.

Now we find all the Saturdays (day 5 of the week) and all Sundays (day 6 of the week) and set them to be freedays as well:

In [5]:
yearDF.loc[(yearDF['Day'].dt.dayofweek == 6)|(yearDF['Day'].dt.dayofweek == 5), 'Freeday'] = True

## Defining trip options

To serve as example trip options, I collected over a thousand trips offered by G Adventures in 2023.  We will go over how I did this programmatically in a future post ;).  Trip data includes the itinerary name, the start day of year, and the end day of year.  I saved this data in a *.csv file that we can import:

In [6]:
tripsDF = pd.read_csv('trips/trips.csv')

I have my *.csv file in a folder called trips so point to that folder and file in this line of code.

# Defining our constraints

## Constraint 1: PTO Day Limit

Recall our first constraint where we can't go over our PTO limit:

\begin{equation}
\mathbf{w}^T \mathbf{A}\mathbf{x} \leq D_{PTO}
\end{equation}

where $\mathbf{w}$ is a vector defining workdays vs. freedays and $\mathbf{A}$ is a matrix defining when trips occur during the year.

Let's create $\mathbf{w}$ using our "Freeday" flag in yearDF below:

In [7]:
w = np.ones(365)
w[yearDF.loc[yearDF['Freeday'], 'Day of Year'] - 1] = 0

Note that we are subtracting one from the "Day of Year" because day of year starts with 1 but our indices for $\mathbf{w}$ start with 0.

Now we create $\mathbf{A}$ :

In [8]:
A = np.zeros((365, len(tripsDF)))
for trip_index, row in tripsDF.iterrows():
    day_range = np.arange(row['start_doy'], row['end_doy'] + 1)
    A[day_range - 1, trip_index] = 1

Here, we iterate through each trip, find its start and end days, and set the corresponding row values in the trip's column to 1.

## Constraint 2: Only 1 Trip at a Time

Recall the second constraint that keeps us from recommending two trips that happen at the same time:

\begin{equation}
\mathbf{A} \mathbf{x} \leq \mathbf{1}_{365}
\end{equation}

Since we already created $\mathbf{A}$ for the first constraint, we just need to create the vector $\mathbf{1}_{365}$:

In [9]:
Ones_365 = np.ones(365)

Simple...

## Constraint 3: No repeats

The third constraint keeps us from choosing the same itinerary multiple times in a year:

\begin{equation}
\mathbf{B} \mathbf{x} \leq \mathbf{1}_{T}
\end{equation}
To construct $\mathbf{B}$, we need to group trips by their itineraries.  Below, we use scikit-learn's LabelEncoder to assign a numerical ID to each itinerary:

In [10]:
itineraryID = LabelEncoder().fit_transform(tripsDF['itinerary_name'])

Next, we iterate through the trips in the tripsDF dataframe to assign each trip to an itinerary as captured by $\mathbf{B}$:

In [11]:
B = np.zeros((tripsDF['itinerary_name'].nunique(), len(tripsDF)))
for index, row in tripsDF.iterrows():
    B[itineraryID[index], index] = 1

Finally, we create a vector of ones that is as along as the number of unique itineraries we have in our trip set:

In [12]:
Ones_T = np.ones(tripsDF['itinerary_name'].nunique())

# Putting it together

In [13]:
data = {}

In [14]:
data['num_vars'] = (w@A).shape[0]

In [15]:
data['constraint_coeffs'] = np.vstack((
    w.T@A,
    A,
    B
))

In [16]:
data['bounds'] = np.hstack((
    20,
    Ones_365,
    Ones_T,
))

In [17]:
data['num_constraints'] = 1 + len(Ones_365) + len(Ones_T)

In [18]:
data['obj_coeffs'] = w.T@A

# Optimization

In [19]:
solver = pywraplp.Solver.CreateSolver('SCIP')

In [20]:
x = {}
for j in range(data['num_vars']):
    x[j] = solver.IntVar(0, 1, 'x[%i]' % j)
print('Number of variables =', solver.NumVariables())

Number of variables = 1513


In [21]:
for i in range(data['num_constraints']):
    constraint = solver.RowConstraint(0, data['bounds'][i], '')
    for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j])
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 420


In [22]:
objective = solver.Objective()
for j in range(data['num_vars']):
    objective.SetCoefficient(x[j], data['obj_coeffs'][j])
objective.SetMaximization()

In [23]:
status = solver.Solve()

In [24]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value(), 'Vacation Days')
    sol_ind = []
    for j in range(data['num_vars']):
        if x[j].solution_value() > 0:
            sol_ind.append(j)
            print(x[j].name(), ' = ', x[j].solution_value(), tripsDF.iloc[j]['itinerary_name'])
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 20.0 Vacation Days
x[126]  =  1.0 Bangkok to Kuta: Summits & Sunsets in Malaysia, Asia - G Adventures

Problem solved in 2281.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 1 branch-and-bound nodes
